In [ ]:
# ==============================================================================
# CELL 1: BOOTSTRAP - INTELLIGENT ENVIRONMENT SETUP
# ==============================================================================
# This cell checks if the required dependencies are already installed. If not,
# it downloads and runs a setup script to prepare the Colab environment.

import sys
import subprocess
import os
import shutil

# --- Configuration ---
# Set to True to install the full OpenStudio CLI (required for running Measures)
INSTALL_CLI = False
# Set the branch of the toolkit to use ('main' for stable, 'dev' for development)
TOOLKIT_BRANCH = 'main'

# --- Verification and Execution Logic ---
try:
    # Attempt to import the core libraries.
    import openstudio
    import openstudio_toolkit
    
    # If INSTALL_CLI is True, also check for the command line interface.
    if INSTALL_CLI and not shutil.which('openstudio'):
        # This will trigger the installation if the CLI is missing but the library is present.
        raise ImportError("OpenStudio CLI not found")

    print("✅ Environment is already set up. Skipping installation.")

except ImportError:
    print("⏳ Required dependencies not found. Proceeding with setup...")
    
    bootstrap_script = 'bootstrap.py'
    # Download the bootstrap script if it doesn't exist OR if it's an empty file.
    if not os.path.exists(bootstrap_script) or os.path.getsize(bootstrap_script) == 0:
        print("--> Downloading bootstrap script...")
        bootstrap_url = f"https://raw.githubusercontent.com/roruizf/OpenStudio-Toolkit/{TOOLKIT_BRANCH}/{bootstrap_script}"
        os.system(f"wget -q -O {bootstrap_script} {bootstrap_url}")

    # Final check to ensure the downloaded script is not empty before running
    if os.path.exists(bootstrap_script) and os.path.getsize(bootstrap_script) > 0:
        print("--> Running environment setup...")
        cli_flag = "--install-cli" if INSTALL_CLI else ""
        os.system(f"python {bootstrap_script} --branch {TOOLKIT_BRANCH} {cli_flag}")
        
        print("\n💥 KERNEL RESTART is likely required for the changes to take effect.")
        print("Please go to 'Runtime' > 'Restart session' in the menu, then run the next cell.")
    else:
        print("❌ ERROR: Failed to download the bootstrap script. Please check the URL and your connection.")

In [ ]:
# ==============================================================================
# CELL 2: RUN TASK
# ==============================================================================
# IMPORTANT: Run this cell AFTER restarting the kernel from Cell 1 if needed.

import os
import pandas as pd

# --- Imports from the Toolkit (now guaranteed to be installed) ---
from openstudio_toolkit.utils.notebook_helpers import get_osm_path
from openstudio_toolkit.utils.osm_utils import load_osm_file_as_model, save_model_as_osm_file
from openstudio_toolkit.tasks.model_setup import rename_subsurfaces_based_on_space_and_surface_names
from openstudio_toolkit.osm_objects.subsurfaces import get_all_subsurface_objects_as_dataframe

# --- User Inputs for File Access ---

# --- FOR GOOGLE COLAB ---
# Set to True to use Google Drive, False to upload a file manually in Colab.
USE_GDRIVE = False
# If using GDrive, provide the full path to your model here.
GDRIVE_OSM_PATH = '/content/drive/MyDrive/path/to/your/model.osm'

# --- FOR LOCAL EXECUTION ---
# This relative path points to the example file from the notebook's location.
LOCAL_OSM_PATH = '../../examples/Cabana-60.osm' 

# --- Get File Path ---
try:
    # The function will automatically use the correct path based on your environment.
    osm_file_path, project_folder = get_osm_path(
        use_gdrive=USE_GDRIVE,
        gdrive_path=GDRIVE_OSM_PATH,
        local_path=LOCAL_OSM_PATH
    )
except (ValueError, FileNotFoundError) as e:
    print(e)
    osm_file_path = None

# --- Main Execution Block ---
if osm_file_path and os.path.exists(osm_file_path):
    
    # Define where the output file will be saved.
    file_name = os.path.basename(osm_file_path)
    output_file_name = f"{os.path.splitext(file_name)[0]}_subsurfaces_renamed.osm"
    output_osm_path = os.path.join(os.getcwd(), output_file_name)
    print(f"INFO: Output will be saved to: {output_osm_path}")

    # 1. Load Model
    osm_model = load_osm_file_as_model(osm_file_path)
    
    # 2. Validate Model
    print("\n--- Validating Model for Task ---")
    report = rename_subsurfaces_based_on_space_and_surface_names.validator(osm_model)
    for msg in report['messages']:
        print(msg)
    print("---------------------------------")
    
    # 3. Run Task and Save
    if report['status'] == 'READY':
        modified_model = rename_subsurfaces_based_on_space_and_surface_names.run(osm_model)
        save_model_as_osm_file(modified_model, output_osm_path)
        print(f"\n✅ Model saved successfully.")

        # 4. Verification Step
        print("\n--- Verifying Results ---")
        print("Loading the modified model to check the new subsurface names...")
        
        verified_model = load_osm_file_as_model(output_osm_path)
        final_subsurfaces_df = get_all_subsurface_objects_as_dataframe(verified_model)
        
        print("Displaying the first 5 subsurfaces from the modified model:")
        display(final_subsurfaces_df.head())

    else:
        print("\n❌ Task did not run. Status:", report['status'])
else:
    if osm_file_path:
        print(f"❌ ERROR: The specified file path does not exist or was not provided: {osm_file_path}")